In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass

rng = np.random.default_rng(seed=1234)


@dataclass
class Data:
    n: int
    m: int
    X: np.ndarray
    y: np.ndarray
    D: np.ndarray


@dataclass
class GDResult:
    topt: np.ndarray
    kopt: int
    K: int
    Jmin: float
    Jval: np.ndarray


def load():
    X = np.loadtxt("nuclear_x.csv", delimiter=",")
    (n, m) = X.shape

    y = np.loadtxt("nuclear_y.csv", delimiter=",")[..., np.newaxis]

    D = np.hstack((X, y))

    rng.shuffle(D, axis=0)

    X = np.hstack((np.ones((n, 1)), D[:, :m]))
    y = D[:, m:]

    return Data(n, m, X, y, D)


data = load()

n, m = (data.n, data.m)
X, y  = (data.X, data.y)

Pw = np.diag((0,) + (1,) * m)

M = y * X
MT = M.T

l = 1e-3
ak = lambda j: 100 / j

t0 = rng.standard_normal((3, 1))
t0[0, 0] = 0.0

In [ ]:
def J(t):
    v = 1 - M @ t
    v *= v > 0
    return np.squeeze(l / 2 * t.T @ Pw @ t + np.average(v))


def Ji(i, t, l):
    yi = y[i]
    xi = X[i : i + 1]
    return np.squeeze(1 / n * (max(0, 1 - yi * xi @ t) + l / 2 * t.T @ Pw @ t))


def G(t, _=None):
    return l * Pw @ t - 1 / n * MT @ (1 - M @ t > 0)


def Gi(i, t):
    yi = y[i]
    xi = X[i : i + 1]
    gr = yi / n * xi.T if np.squeeze(yi * xi @ t) < 1 else 0
    return l / n * Pw @ t - gr


def GD(t0, G, K=10000):
    t = np.copy(t0)

    kopt = 0
    Jmin = np.inf
    topt = np.zeros_like(t)

    Jval = []
    Jval.append(J(t))

    for k in range(1, K + 1):
        t -= ak(k) * G(t, k)
        Jk = J(t)
        if Jk < Jmin:
            kopt = k
            Jmin = Jk
            np.copyto(src=t, dst=topt)
        Jval.append(Jk)

    return GDResult(
        topt=np.squeeze(topt),
        kopt=kopt,
        K=K,
        Jmin=Jmin,
        Jval=Jval,
    )


def plot_classes(ax, data, topt):
    D = data.D
    n, m = data.n, data.m
    class1 = D[D[:, m] == 1][:, :m].T
    class2 = D[D[:, m] == -1][:, :m].T
    x = D[:, 0]
    xmin, xmax = (np.min(x), np.max(x))
    x = np.linspace(xmin, xmax)
    [t0, t1, t2] = topt

    ax.scatter(class1[0], class1[1], color="red", s=1., label="Class 1")
    ax.scatter(class2[0], class2[1], color="blue", s=1., label="Class 2")
    ax.plot(x, -1 / t2 * (t1 * x + t0), color="black", label="Decision Boundary")
    ax.set_xlabel(r"$x_1$")
    ax.set_ylabel(r"$x_2$")
    ax.legend()
    return ax

In [ ]:
res = GD(t0, G, K=1000)

In [ ]:
fig, [ax1, ax2] = plt.subplots(1,2)


ax2.scatter(np.arange(res.K+1), np.log10(res.Jval), marker="x", s=1.)
ax2.set_xlabel(r"$k$")
ax2.set_ylabel(r"$\log_{10} J(\theta_k)$")

plot_classes(ax1, data, res.topt)


In [ ]:
m = np.diag((2, -1, -2))
m[m[:,0] > 0]
